In [1]:
import pandas as pd
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import requests
import urllib3
import time
import pandas as pd
from tqdm import tqdm
import openai

ssl._create_default_https_context = ssl._create_unverified_context
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
%store -r voter_default

In [3]:
#행에 맞게 수정
voter = voter_default[600:]

뉴스 크롤링 진행

In [39]:
news_list = []
for i in range(len(voter)):
    news_list.append([])

In [40]:
#url 가져오는 함수
def get_url(x):
    global url_news
    if '학교' in voter.iloc[n, 1] and voter.iloc[n, 3] == '교수':
        url_news = f'https://search.naver.com/search.naver?where=news&query={voter.iloc[n, 1].replace("학교", "")}+"{voter.iloc[n, 0]}+교수"{x}'
    elif voter.iloc[n, 2] == '교수':
        url_news = f'https://search.naver.com/search.naver?where=news&query={voter.iloc[n, 1]}+"{voter.iloc[n, 0]}+교수"'
    else:
        url_news = f'https://search.naver.com/search.naver?where=news&query="{voter.iloc[n, 1]}+{voter.iloc[n, 0]}"'
    res = requests.get(url_news, headers=headers, verify=False).content
    global news_soup
    news_soup = BeautifulSoup(res, "html.parser")

In [41]:
#뉴스기사 크롤링
for n in range(len(voter)):
    %store news_list
    get_url('')
    news_num = len(news_soup.select('#main_pack > section.sc_new.sp_nnews._prs_nws > div > div.group_news > ul > li', class_="bx"))
    if news_num > 3:
        try:
            nd = 1 # 뉴스 순서 (sp_nws)에 들어갈 숫자
            nt = 0 # nt가 2가 되면(기사 2개 가져오면) stop
            while nt < 2:
                try:
                    news_date = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_info > div.info_group > span').text
                    news_title = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_contents > a.news_tit').text
                    if '부고' in news_title:
                        nd += 1
                        continue
                    news1 = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_contents > a').attrs['href']
                    res = requests.get(news1, headers=headers, verify=False).content
                    news1_soup = BeautifulSoup(res, "html.parser")
                    news1_txt = re.sub(r'\n|\t|\r', '', news1_soup.text)
                    news_list[n].append(news_title)
                    news_list[n].append(news_date)
                    news_list[n].append(news1)
                    news_list[n].append(news1_txt)
                    nt += 1
                    nd += 1
                except:
                    nd += 1

            get_url('&sort=1')

            nd = 1 # 뉴스 순서 (sp_nws)에 들어갈 숫자
            nt = 0 # nt가 2가 되면(기사 2개 가져오면) stop
            while nt < 2:
                if nd > 15:
                    break
                try:
                    news_date = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_info > div.info_group > span').text
                    news_title = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_contents > a.news_tit').text
                    if news_title in news_list[n] or news_title in news_list[n]:
                        nd += 1
                        continue
                    if '부고' in news_title:
                        nd += 1
                        continue
                    news1 = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_contents > a').attrs['href']
                    if news1 in news_list[n] or news1 in news_list[n]:
                        nd += 1
                        continue
                    res = requests.get(news1, headers=headers, verify=False).content
                    news1_soup = BeautifulSoup(res, "html.parser")
                    news1_txt = re.sub(r'\n|\t|\r', '', news1_soup.text)
                    news_list[n].append(news_title)
                    news_list[n].append(news_date)
                    news_list[n].append(news1)
                    news_list[n].append(news1_txt)
                    nt += 1
                    nd += 1
                except:
                    nd += 1
        except Exception as ex:
            print(f'{voter.iloc[n, 1]+" "+voter.iloc[n, 0]}', ex)
    else:
        try:
            for num in range(news_num):
                news_date = news_soup.select_one(f'#sp_nws{num+1} > div > div > div.news_info > div.info_group > span').text
                news_title = news_soup.select_one(f'#sp_nws{num+1} > div > div > div.news_contents > a.news_tit').text
                news1 = news_soup.select_one(f'#sp_nws{num+1} > div > div > div.news_contents > a').attrs['href']
                res = requests.get(news1, headers=headers, verify=False).content
                news1_soup = BeautifulSoup(res, "html.parser")
                news1_txt = re.sub(r'\n|\t|\r', '', news1_soup.text)
                news_list[n].append(news_title)
                news_list[n].append(news_date)
                news_list[n].append(news1)
                news_list[n].append(news1_txt)
        except Exception as ex:
            print(f'{voter.iloc[n, 1]+" "+voter.iloc[n, 0]}', ex)
    print(url_news)
    print(news_num)

Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=중앙대+"임병하+교수"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query="St. Marys University+임성배"
4
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=고려대 +"임성택+교수"&sort=1
10
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=상지대+"임세헌+교수"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=연세대+"임일+교수"&sort=1
10
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query="한국오라클(유)+임재수"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=아주대 +"임재익+교수"&sort=1
4
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=충남대+"임학빈+교수"
3
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query="단국대학교+장기태"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=서울시립대 +"장남식+교수"&

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://search.naver.com/search.naver?where=news&query=서울대+"장정주+교수"&sort=1
7
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query="(주)이브로드캐스트+장철웅"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=중앙대+"장항배+교수"&sort=1
10
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query="행정자치부+장희정"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query="대구산업정보대학+전광진"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=상명대 +"전기정+교수"&sort=1
10
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query="(주)STS 엔터프라이즈+전대식"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=유한대 +"전동원+교수"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=강남대 +"전병찬+교수"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=가천대+"전성민+교수"&sort=1
10
Stored 'news_list' (

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://search.naver.com/search.naver?where=news&query=서울대+"조신섭+교수"&sort=1
7
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=청주대 +"조용길+교수"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=동아대+"조용복+교수"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=서울대+"조우제+교수"
1
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=국민대 +"조윤호+교수"
2
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query="금융결제원 +조이남"
1
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=광운대+"조재희+교수"
3
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=백석문화대학+"조정융+교수"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query="한국과학기술원+조항정"
0
Stored 'news_list' (list)
https://search.naver.com/search.naver?where=news&query=울산대 +"조희연+교수"
1
Stored 'news_list' (list)
https://search.naver.com/s

뉴스기사 글자 수 줄이가(10000자 이내로)

In [42]:
for i in range(len(news_list)):
    if len(news_list[i]) > 0:
        for j in range(1, int(len(news_list[i])/4+1), 1):
            if news_list[i][j*4-1].find('ⓒ') != -1:
                ni = news_list[i][j*4-1].find('ⓒ')
                news_list[i][j*4-1] = news_list[i][j*4-1][:ni]
            elif news_list[i][j*4-1].find('ⓒ') == -1 and news_list[i][j*4-1].find('right') != -1:
                ni = news_list[i][j*4-1].find('right')
                news_list[i][j*4-1] = news_list[i][j*4-1][:ni]
            else:
                news_list[i][j*4-1] = news_list[i][j*4-1]
            if len(news_list[i][j*4-1]) > 10000:
                news_list[i][j*4-1] = news_list[i][j*4-1][:10000]
                print(i)

18
33
56
58
58
82
82
83
151
156
163
209
209
209
216


---

In [43]:
news_df = pd.DataFrame(news_list, columns = ['제목_1', '날짜_1', '링크_1', '내용_1', '제목_2', '날짜_2', '링크_2', '내용_2', '제목_3', '날짜_3', '링크_3', '내용_3', '제목_4', '날짜_4', '링크_4', '내용_4'])

In [44]:
news_list_800 =  news_list
news_df_800 = news_df

In [45]:
%store news_list_800
%store voter
%store news_df_800

Stored 'news_list_800' (list)
Stored 'voter' (DataFrame)
Stored 'news_df_800' (DataFrame)


In [46]:
news_df = news_df_800
news_list = news_list_800

GPT 호출 준비

In [4]:
voters = voter.drop(['임시직위'], axis = 1)
voters.index = [x for x in range(218)]

In [7]:
news_df =  news_df_800
news_list = news_list_800

In [8]:
df = pd.concat([voters, news_df], axis=1, join= 'outer')

In [10]:
#행에 맞게 수정
%store -r profile_df_default
profile_df = profile_df_default[600:]
profile_df.index = [x for x in range(218)]

In [ ]:
d
# 발급받은 API 키 설정
OPENAI_API_KEY =
openai.api_key = OPENAI_API_KEY

In [12]:
#ChatGPT로 프롬프트 보냄
#"gpt-3.5-turbo-16k"
#"gpt-4"
def get_completion(x, temperature=0):
    model = x
    messages = prompt
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

뉴스기사 요약

In [13]:
#뉴스기사 요약하는 함수
#w: 날짜, #x: 이름, y: 기사, z: 소속
def get_news(w, x, y, z):
  global p_news
  p_news = f"""
      Message will be written in Korean.
      Based on the news, wirte down what {x} did. Please write it in 1~2 sentences.
      Add {w} in front of the sentence if you can.
      Please judge if {x} in the news belongs to {z}. If not, do not write the senteces and the date, Write 'X' only.
      - It is likely that {x} belongs to {z} if you can find {z} and {x} together in one sentence.
      If the news does not contain what {x} did, write 'X' only. In this case, you don't need to add {w}
      """

  global news
  news = y

  global prompt
  prompt = [{"role" : "system",
              "content" : p_news},
            {"role" : "user",
            "content" : f'{news}'}]
  return prompt

In [53]:
gpt_news = []
for i in range(len(voters)):
    gpt_news.append([])

In [14]:
%store -r gpt_news

In [18]:
gpt_news[216]

['2023.04.24. 황현석 교수는 중소벤처기업부에서 공모하는 예비 창업 패키지 사업과 창업 중심 대학 예비 창업자 지원 사업에 선정되었다.',
 "21면 1단 한림대 문성훈·황현석 교수 정부사업 선정: 한림대 경영학과 문성훈 교수는 창업중심대학 초기 창업기업 지원사업에 '해외주식 투자자를 위한 맞춤형 택스테크 서비스'로 선정되었고, 황현석 교수는 중소벤처기업부 예비창업패키지 사업과 창업중심대학 예비창업자 지원 사업에 선정되었다.",
 '2022.09.23. 한림대학교는 경영학과 교수인 황현석을 대외협력처장으로 임명했다.',
 '2022.09.23. 한림대학교는 경영학과 황현석 교수를 신임 대외협력처장으로 임명했다.']

In [19]:
#각 뉴스기사들 요약
for n in range(len(news_list)):
    print(n)
    %store gpt_news
    if len(news_list[n]) > 0:
        for j in range(1, int(len(news_list[n])/4+1), 1):
            try:
                print(j)
                get_news(news_list[n][j*4-3], voters.iloc[n, 0], news_list[n][j*4-1], voters.iloc[n, 1])
                gpt_news[n].append(get_completion("gpt-3.5-turbo-16k"))
                if len(gpt_news[n]) > 4:
                    continue
            except:
                print('retry in 30s')
                get_news(news_list[n][j*4-3], voters.iloc[n, 0], news_list[n][j*4-1], voters.iloc[n, 1])
                gpt_news[n].append(get_completion("gpt-3.5-turbo-16k"))
                if len(gpt_news[n]) > 4:
                    continue
        print(gpt_news[n])
        print('-------------------------------------------------------')
    if n == 25 or n == 50 or n == 75 or n == 100 or n == 125 or n == 150 or n == 175 or n == 200:
        time.sleep(30)


217
Stored 'gpt_news' (list)


In [20]:
gn_df = pd.DataFrame(gpt_news, columns = ['기사요약_1', '기사요약_2', '기사요약_3', '기사요약_4'])

In [21]:
profile_df = profile_df.drop(['이름', '소속', '직위'], axis=1)

In [22]:
test_df = pd.concat([df, gn_df, profile_df], axis = 1, join = 'outer')

In [23]:
test_df = test_df[['이름', '소속', '직위', '핸드폰', '이메일', '세부소속', '위치', '학력', '관심분야', '기타 키워드',
                   '제목_1', '날짜_1', '링크_1', '내용_1', '기사요약_1', '제목_2',
       '날짜_2', '링크_2', '내용_2', '기사요약_2', '제목_3', '날짜_3', '링크_3', '내용_3', '기사요약_3', '제목_4', '날짜_4',
       '링크_4', '내용_4', '기사요약_4']]


In [24]:
test_df = test_df.fillna('X')

In [25]:
test_df.to_excel('크롤링 정보(~818)).xlsx')

In [26]:
#test_df = pd.read_excel('크롤링 정보(상위 500).xlsx')

In [26]:
needed = ['소속', '직위', '세부소속', '관심분야', '기타 키워드', '기사요약_1', '기사요약_2', '기사요약_3', '기사요약_4']
y_data = []
for i in range(len(test_df)):
    y_data.append([])

In [27]:
for i in range(len(test_df)):
    for j in needed:
        y_data[i].append(test_df.loc[i, j])

In [28]:
#요약된 기사들 재요약
#x: 이름, y: 정보(프로필&뉴스기사 요약), z: 학력
def get_newsorg(x, y, z):
    global summarize
    summarize = f"""
        아래의 조건들을 준수하여, 주어진 text로부터 {x}에 대한 정보들을 작성하라.
        1. 주어진 text로부터 {x}의 직함과 업적을 단어의 중복없이 정보의 손실없이, 개조식으로(서술형으로 하지 말고) 요약하라.
        - 직함을 적을 때, '겸임교수', '부교수', '조교수' 등 '교수'의 종류일 경우 구분없이 '교수'로 통일한다
        2. 『』기호 안의 학력이 only '카이스트', 'KAIST', '한국과학기술원', '서울대학교' 4가지 중 하나에 해당할 경우에만, 
            '★' 표시와 함께 그가 해당 학력임을 작성하라.
            The word in『』is his/her academic background. Do not mark '★' if his/her academic background is not '카이스트', 'KAIST',
            '한국과학기술원' or '서울대학교 
        3. {x}가 인공지능 혹은 AI와 관련된 사람일 경우, '☆' 표시와 함께 그가 인공지능 혹은 AI에 관심이 있음을 작성하고,
            가능하다면 그 이유를 작성하라.
            인공지능 혹은 AI와 관련되지 않은 사람일 경우 굳이 문장을 적지 마라. Do not write sentences or '☆' if she/he is not related with AI.
        4. 업적 작성 시, 가능하다면 문장 뒤에 날짜를 기재하라.
        5. 주어진 text만 참고하여 작성하라. 없는 정보를 지어내서는 안 된다. 정보나 업적이 없을 경우 기재하지 않아도 된다.
            [] 기호나 'X'를 써서는 안 된다.
        6. 업적 중 날짜만 알 수 있는 건에 대해, 날짜와 함께 "뉴스기사 확인 요망"을 적어라.
        7. 이름을 적을 필요는 없다.
        8. 아래와 같은 형식에 따라 한국어로 작성하라

        - 
        - 
        - 
        ...

        """
    
    global sum_news
    sum_news = y

    global bg
    bg = z

    global prompt
    prompt = [{"role" : "system",
                "content" : summarize},
                {"role" : "user",
                "content" : f'{sum_news}'},
                {"role" : "user",
                "content" : f'{bg}'}]
    
    return prompt

In [31]:
gpt_final =  []
for i in range(len(test_df)):
    gpt_final.append([])

In [32]:
for n in range(len(test_df)):
    print(n)
    %store gpt_final
    try:
        get_newsorg(voters.iloc[n, 0], y_data[n], f'『{test_df.iloc[n, 7]}』')
        gpt_final[n].append(get_completion("gpt-4"))
        print(gpt_final[n][0])
        print('---------------------------------------------')
    except openai.error.InvalidRequestError:
        print('InvalidRequestError')
        y_data[n][8] = y_data[n][8][:3500]
        get_newsorg(voters.iloc[n, 0], y_data[n], f'『{test_df.iloc[n, 7]}』')
        gpt_final[n].append(get_completion("gpt-4"))
        print(gpt_final[n][0])
        print('---------------------------------------------')
    except:
        print('30s wait')
        time.sleep(30)
        get_newsorg(voters.iloc[n, 0], y_data[n], f'『{test_df.iloc[n, 7]}』')
        gpt_final[n].append(get_completion("gpt-4"))
        print(gpt_final[n][0])
        print('---------------------------------------------')
    if n == 25 or n == 50 or n == 75 or n == 100 or n == 125 or n == 150 or n == 175 or n == 200:
        time.sleep(30)

0
Stored 'gpt_final' (list)
- 중앙대학교 교수
- 경영경제대학 소속
- 경영정보시스템, e-Business 전공
---------------------------------------------
1
Stored 'gpt_final' (list)
- 교수
- 학과장
- Faceboo 업적 (2021.04.29.)
- 뉴스기사 확인 요망 (2021.04.28.)
---------------------------------------------
2
Stored 'gpt_final' (list)
- 고려대학교 교수
- 경상대학 경영정보학과
- 경영정보학 전공
- 디지털컨버전스포럼 간사
- 모바일포럼 경제산업분과 간사
---------------------------------------------
3
Stored 'gpt_final' (list)
- 교수
- 상지대학교
- 경영정보학과
- MIS 전문가
- 경영 관련 다학제 분야 전문가
---------------------------------------------
4
Stored 'gpt_final' (list)
- 교수
- 경영대학
- Management information systems
- Machine learning
- 한국지능정보시스템 학회 회장
- 한국자동차산업학회 부회장
- 초거대 인공지능 '챗GPT'에 대응하기 위한 한국 기업들의 전략 분석 (2023.02.17)
- 서울경제TV '수축경제 포럼'에서 구독경제의 해법 제시 (2020.02.24)
- 경기도의회 정보화위원회의 의정정보화 연찬회에서 '디지털 리더십' 특강 실시 (2022.12.20)
- ☆ 인공지능에 관심이 있음. 이유: 초거대 인공지능 '챗GPT'에 대한 전략 분석을 통해 인공지능에 대한 이해와 관심을 보여줌
---------------------------------------------
5
Stored 'gpt_final' (list)
- 한국오라클(유) 근무
- 상무
- 금융사업 관련 업무 수행
------

In [33]:
#index에 따라 수정
adf =pd.DataFrame(gpt_final)
#adf.index = [x for x in range(100, 200 , 1)]

In [34]:
test_df.insert(3, '프로필', adf)

In [35]:
len(test_df.columns)

31

In [36]:
test_df.to_excel('유권자정보(~818).xlsx')

In [37]:
gpt_news_800 = gpt_news
gpt_final_800 = gpt_final
test_df_800 = test_df

In [38]:
%store gpt_news_800
%store gpt_final_800
%store test_df_800

Stored 'gpt_news_800' (list)
Stored 'gpt_final_800' (list)
Stored 'test_df_800' (DataFrame)


: 